# Sheath Expansion Simulations Results - Analysis Round 2
First from a series of notebooks analysing the sheath expansions simulations in a manner similar to Bergmann in his 2002 paper. This one analyses a previously combined xarray dataset

In [70]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.utils as spu

In [91]:
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [3]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [4]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [5]:
skippable_scans = set() 
single_sims = set()

In [59]:
# scans_searchstr = '*/*/sheath_exp/*'
scans_searchstr = '*/*/new_sheath_exp/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)



scans = list(scans)
scans.sort()

In [61]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: marconi/spice2/new_sheath_exp/angled_flush
	[0,0]: angled_flush/alpha_yz_-10.0
	[0,1]: angled_flush/alpha_yz_-12.0
	[0,2]: angled_flush/alpha_yz_-2.0
	[0,3]: angled_flush/alpha_yz_-20.0
	[0,4]: angled_flush/alpha_yz_-3.0
	[0,5]: angled_flush/alpha_yz_-30.0
	[0,6]: angled_flush/alpha_yz_-4.0
	[0,7]: angled_flush/alpha_yz_-5.0
	[0,8]: angled_flush/alpha_yz_-6.0
	[0,9]: angled_flush/alpha_yz_-8.0
[1]: marconi/spice2/new_sheath_exp/angled_recessed
	[1,0]: angled_recessed/alpha_yz_-10.0
	[1,1]: angled_recessed/alpha_yz_-12.0
	[1,2]: angled_recessed/alpha_yz_-2.0
	[1,3]: angled_recessed/alpha_yz_-20.0
	[1,4]: angled_recessed/alpha_yz_-3.0
	[1,5]: angled_recessed/alpha_yz_-30.0
	[1,6]: angled_recessed/alpha_yz_-4.0
	[1,7]: angled_recessed/alpha_yz_-5.0
	[1,8]: angled_recessed/alpha_yz_-6.0
	[1,9]: angled_recessed/alpha_yz_-8.0
[2]: marconi/spice2/new_sheath_exp/flat_flush
	[2,0]: flat_flush/alpha_yz_-10.0
	[2,1]: flat_flush/alpha_yz_-12.0
	[2,2]: flat_flush/alpha_yz_-2.0
	[2,3]: flat_flu

In [60]:
spl_path = spice_dir / all_run_dirs[scans[0]][1]
print(spl_path)

/home/jleland/data/external_big/spice/marconi/spice2/new_sheath_exp/angled_flush/alpha_yz_-12.0


In [8]:
# probe_desig = {
#     'angled_flush': {'theta_p':10.0, 'recession':0.0},
#     'angled_semi-recessed': {'theta_p':10.0, 'recession':5.0e-4},
#     'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3},
#     'semi-angled_flush': {'theta_p':5.0, 'recession':0.0},
#     'semi-angled_semi-recessed': {'theta_p':5.0, 'recession':5.0e-4},
#     'semi-angled_recessed': {'theta_p':5.0, 'recession':1.0e-3},
#     'flat_flush': {'theta_p':0.0, 'recession':0.0},
#     'flat_semi-recessed': {'theta_p':0.0, 'recession':5.0e-4},
#     'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3},
# }

# cb_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
#               '#f781bf', '#a65628', '#984ea3', 
#               '#999999', '#e41a1c', '#dede00']

probe_desig = {
    'angled_flush': {'theta_p':10.0, 'recession':0.0},
#     'angled_semi-recessed': {'theta_p':10.0, 'recession':5.0e-4},
    'angled_recessed': {'theta_p':10.0, 'recession':1.0e-3},
#     'semi-angled_flush': {'theta_p':5.0, 'recession':0.0},
#     'semi-angled_semi-recessed': {'theta_p':5.0, 'recession':5.0e-4},
#     'semi-angled_recessed': {'theta_p':5.0, 'recession':1.0e-3},
    'flat_flush': {'theta_p':0.0, 'recession':0.0},
#     'flat_semi-recessed': {'theta_p':0.0, 'recession':5.0e-4},
    'flat_recessed': {'theta_p':0.0, 'recession':1.0e-3},
}

cb_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
              '#f781bf', '#a65628', '#984ea3', 
              '#999999', '#e41a1c', '#dede00']


probe_colour = {probe_label: cb_palette[i] for i, probe_label in enumerate(probe_desig.keys())}
probe_colour

{'angled_flush': '#377eb8',
 'angled_recessed': '#ff7f00',
 'flat_flush': '#4daf4a',
 'flat_recessed': '#f781bf'}

In [9]:
def decompose_new_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (((L + g) * np.tan(theta)) 
                                       + (L * np.tan(theta_p)) - d_perp))
    x = np.tan(theta) + (2 * np.tan(theta_p))
    return x, y

def decompose_alt_new_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (L + g + (((L * np.tan(theta_p)) - d_perp) / np.tan(theta))))
    x = np.cos(theta) / np.sin(theta)
    return x, y

def decompose_2d_box_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    L_eff = (L/np.cos(theta_p)) - ((d_perp - (g * np.tan(theta))) 
                                   / ((np.cos(theta_p) * np.tan(theta)) + np.sin(theta_p)))
    
    y = a * np.sqrt(np.sin(theta + theta_p)) * L_eff
    x = np.cos(theta + theta_p) / np.sin(theta + theta_p)
    return x, y

def decompose_sheath_exp_param(a, theta, L, g, d_perp=0, theta_p=0):
    y = a * (L + g) * np.sqrt(np.sin(theta))
    x = np.cos(theta) / np.sin(theta)
    return x, y


In [10]:
splopter =  spl.Splopter(spl_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True)

parser = inp.InputParser(input_filename=spl_path / 'input.inp')
denormaliser = nrm.Denormaliser(dt=splopter.tdata.dt, input_parser=parser) #, dimensions=3)


Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp/angled/alpha_yz_-2.0) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [11]:
dV = parser.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = parser.getfloat('mks', 'mks_te')
n_0 = parser.getfloat('mks', 'mks_n0')

L = parser.getfloat('rectangle2', 'yhigh') - parser.getfloat('rectangle2', 'ylow')
g = parser.getfloat('rectangle2', 'ylow') - parser.getfloat('rectangle0', 'yhigh') 
lambda_D = lpu.debye_length(T_e, n_0)
theta_p = np.radians(10.0)

print(f"L = {L} L_d \n"
      f"g = {g} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta_p = {theta_p} \n")

L = 300.0 L_d 
g = 60.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta_p = 0.17453292519943295 



In [12]:
# combined_ds = xr.load_dataset('sheath_exp_ivs.nc')
combined_ds = xr.load_dataset('new_sheath_exp_ivs.nc')
combined_ds

<xarray.Dataset>
Dimensions:               (recession: 2, theta: 9, theta_p: 2, voltage: 401)
Coordinates:
  * recession             (recession) float64 0.0 0.001
    theta_p_desc          (theta_p) object 'flat' 'angled'
  * theta_p               (theta_p) float64 0.0 10.0
  * theta                 (theta) float64 3.0 4.0 5.0 6.0 ... 12.0 20.0 30.0
  * voltage               (voltage) float64 -9.95 -9.9 -9.85 ... 9.95 10.0 10.05
    recession_desc        (recession) object 'flush' 'recessed'
    theta_p_rads          (theta_p) float64 0.0 0.1745
    theta_rads            (theta) float64 0.05236 0.06981 ... 0.3491 0.5236
Data variables:
    str_iv_chi2           (recession, theta_p, theta) float64 6.743e+03 ... 159.0
    str_iv_reduced_chi2   (recession, theta_p, theta) float64 43.5 ... 1.026
    str_iv_I_sat          (recession, theta_p, theta) float64 -11.35 ... -71.91
    str_iv_d_I_sat        (recession, theta_p, theta) float64 3.13 ... 1.296
    str_iv_a              (recession, theta_p, theta) float64 1.868e-15 ... 0.04223
    str_iv_d_a            (recession, theta_p, theta) float64 0.12 ... 0.005092
    str_iv_T_e            (recession, theta_p, theta) float64 1.995 ... 0.9402
    str_iv_d_T_e          (recession, theta_p, theta) float64 0.3035 ... 0.02061
    str_iv_V_f            (recession, theta_p, theta) float64 -4.721 ... -2.776
    str_iv_d_V_f          (recession, theta_p, theta) float64 0.05557 ... 0.006002
    str_iv_run_dir        (recession, theta_p, theta) object 'marconi/spice2/new_sheath_exp/flat_flush/alpha_yz_-3.0' ... 'marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-30.0'
    str_iv_voltage_min    (recession, theta_p, theta) float64 -9.95 ... -9.95
    str_iv_voltage_max    (recession, theta_p, theta) float64 -2.05 ... -2.05
    norm_iv_chi2          (recession, theta_p, theta) float64 138.3 ... 153.5
    norm_iv_reduced_chi2  (recession, theta_p, theta) float64 1.281 ... 1.073
    norm_iv_I_sat         (recession, theta_p, theta) float64 -6.476 ... -72.86
    norm_iv_d_I_sat       (recession, theta_p, theta) float64 0.1268 ... 0.7461
    norm_iv_a             (recession, theta_p, theta) float64 0.1512 ... 0.04027
    norm_iv_d_a           (recession, theta_p, theta) float64 0.01033 ... 0.003633
    norm_iv_voltage_min   (recession, theta_p, theta) float64 -5.454 ... -7.219
    norm_iv_voltage_max   (recession, theta_p, theta) float64 -0.004398 ... -0.01927
    ion_chi2              (recession, theta_p, theta) float64 127.9 ... 196.4
    ion_reduced_chi2      (recession, theta_p, theta) float64 1.184 ... 1.373
    ion_I_sat             (recession, theta_p, theta) float64 6.465 ... 79.16
    ion_d_I_sat           (recession, theta_p, theta) float64 0.04734 ... 0.4406
    ion_a                 (recession, theta_p, theta) float64 0.1515 ... 0.01446
    ion_d_a               (recession, theta_p, theta) float64 0.004415 ... 0.00207
    ion_voltage_min       (recession, theta_p, theta) float64 0.01708 ... 0.05173
    ion_voltage_max       (recession, theta_p, theta) float64 3.569 ... 4.404
    current               (recession, theta_p, theta, voltage) float64 -10.29 ... 915.7
    current_e             (recession, theta_p, theta, voltage) float64 -0.02646 ... -918.8
    current_i             (recession, theta_p, theta, voltage) float64 10.32 ... 3.105
    d_current             (recession, theta_p, theta, voltage) float64 0.2479 ... 6.614
    d_current_e           (recession, theta_p, theta, voltage) float64 0.01171 ... 6.695
    d_current_i           (recession, theta_p, theta, voltage) float64 0.2477 ... 0.495
    time                  (recession, theta_p, theta, voltage) float64 0.0 ... 2.581

In [13]:
lowdens_dir = pth.Path('/home/jleland/data/external/spice/')
fit_data_filename = 'lowdens_fitdata.csv'

fit_df = pd.read_csv(lowdens_dir / fit_data_filename, index_col=0)
fit_df

,probe,angle,temp,d_temp,isat,d_isat,a,d_a,v_f,d_v_f
0,angled,1.0,1.025535,0.020026,3.283136,0.082634,0.034400,0.008213,-3.372417,0.007165
1,angled,2.0,1.045895,0.016761,7.148155,0.163483,0.013990,0.007510,-3.526380,0.006954
2,angled,3.0,1.048542,0.008816,11.100458,0.129524,0.004756,0.003758,-3.484159,0.003447
3,angled,4.0,1.023927,0.018306,14.362077,0.343829,0.012245,0.007634,-3.423099,0.007020
4,angled,5.0,1.013085,0.010332,18.032248,0.256156,0.010595,0.004569,-3.457255,0.004239
...,...,...,...,...,...,...,...,...,...,...
83,sprobe,30.0,1.033611,0.010399,62.740506,0.552424,0.012586,0.002549,-2.732093,0.002039
84,sprobe,45.0,1.047113,0.014941,83.666200,1.018863,0.007753,0.003510,-2.690400,0.002807
85,sprobe,60.0,1.006582,0.010704,86.495626,0.719475,0.016005,0.002336,-2.594291,0.002069
86,sprobe,75.0,0.996101,0.012401,87.772275,0.795372,0.017997,0.002498,-2.483807,0.002520


In [14]:
flush_df = fit_df.loc[fit_df['probe'] == 'flush']
flush_df = flush_df.loc[flush_df['angle'] <= 30].drop(20).rename(columns={'angle':'theta'})

flush_df['theta_rads'] = np.radians(flush_df['theta'])
flush_df['theta_p'] = 0.0
flush_df['recession'] = 0.0
flush_df['theta_p_rads'] = np.radians(flush_df['theta_p'])

flush_ds = flush_df.to_xarray().swap_dims({'index':'theta'})

## Let's do some plotting!

In [15]:
dummy_theta = np.linspace(0.00001, 45.0, 5000)
flush_calced_a = lpu.calc_sheath_expansion_coeff(
    T_e,
    n_0,
    L * lambda_D,
    g * lambda_D, 
    np.radians(dummy_theta),
    c_1=0.5,
    c_2=0.6,
)



In [16]:
# cb_palette = ['#4477AA', '#66CCEE', '#228833', '#CCBB44', '#EE6677', '#AA3377', '#BBBBBB']


In [112]:
probe='angled_recessed'
plot_ds = combined_ds.sel(**probe_desig[probe])

fig, ax = plt.subplots()

plot_ds['str_iv_a'].plot.line(x='theta', ax=ax, marker='x', mfc='none', linestyle='none', label='Straight')
plot_ds['norm_iv_a'].plot.line(x='theta', ax=ax, marker='s', mfc='none', linestyle='none', label='Norm IV')
plot_ds['ion_a'].plot.line(x='theta', ax=ax, marker='o', mfc='none', linestyle='none', label='Ion')

calced_a = lpu.calc_new_sheath_expansion_param(
    T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
    plot_ds.recession.values, plot_ds.theta_p_rads.values, c_1=0.5, c_2=0.6,
)

ax.errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', color='r', linewidth=0.8, alpha=0.6)
# ax.errorbar(dummy_theta, flush_calced_a, label=r'calc', fmt='-', color='b', linewidth=0.8, alpha=0.6)
ax.set_ylim(1e-3, 5e-1)
ax.set_xlim(-0.5, 30.5)

ax.legend()

In [44]:
fig, ax = plt.subplots()

# probes_oi = [p for p in probe_desig.keys() if 'recessed' not in p]
# probes_oi = [p for p in probe_desig.keys() if 'flat' in p]
probes_oi = [p for p in probe_desig.keys()]

for i, probe in enumerate(probes_oi):
    plot_ds = combined_ds.sel(**probe_desig[probe])

#     plot_ds['str_iv_a'].plot.line(x='theta', ax=ax, marker='x', mfc='none', linestyle='none', label='Straight')
#     plot_ds['norm_iv_a'].plot.line(x='theta', ax=ax, marker='s', mfc='none', linestyle='none', label=f'{probe} - norm',
#                                    color=cb_palette[2*i])
#     plot_ds['ion_a'].plot.line(x='theta', ax=ax, marker='o', mfc='none', linestyle='none', label=f'{probe} - ion', 
#                                color=cb_palette[2*i])
    ax.errorbar(plot_ds['theta'].values, plot_ds['ion_a'].values, yerr=plot_ds['ion_d_a'].values, marker='o', mfc='none', linestyle='none', 
                label=f'{probe}', color=cb_palette[2*i], ecolor=cb_palette[2*i])

    calced_a = lpu.calc_new_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values, c_1=0.9, c_2=0.9,
    )

    ax.errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', color=cb_palette[2*i], linewidth=0.8, alpha=0.6)
    # ax.errorbar(dummy_theta, flush_calced_a, label=r'calc', fmt='-', color='b', linewidth=0.8, alpha=0.6)
    ax.set_ylim(1e-3, 2.5e0)
    ax.set_xlim(-0.5, 30.5)

    ax.legend()

In [116]:
fig, axes = plt.subplots(2, 2, figsize=[8,8], sharex=True)

probes_oi = [p for p in probe_desig.keys()]

probe_old_cs = [
    (0.09, 5.6),
    (-0.031, 5.2),
    (0.44, 6.3),
    (3.8, -4.0)
]
probe_new_cs = [
    (-0.24, 3.5),
    (0.76, 2.3),
    (0.8, 3.9),
    (3, 0.5)
]
probe_2db_cs = [
    (1, 3.9),
    (2.4, 1.9),
    (0.44, 6.3),
    (3.8, -4.0)
]


for i, probe in enumerate(probes_oi):
    ax = axes[i%2][i//2]
    plot_ds = combined_ds.sel(**probe_desig[probe])

    ax.errorbar(plot_ds['theta'].values, plot_ds['ion_a'].values, yerr=plot_ds['ion_d_a'].values, marker='o', mfc='none', linestyle='none', 
                label=f'{probe}', color=cb_palette[i], ecolor=cb_palette[i])
#     c_1 = 0.9
#     c_2 = 0.6

#     c_1 = 6
#     c_2 = 0.6

#     c_1 = 2.4
#     c_2 = 1.9

#     c_1 = 0.76
#     c_2 = 2.3

    c_2_o, c_1_o = probe_old_cs[i]
    c_2_n, c_1_n = probe_new_cs[i]
    c_2_b, c_1_b = probe_2db_cs[i]
    calced_a = lpu.calc_sheath_expansion_coeff(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        c_1=c_1_o, c_2=c_2_o
    )
    
    calced_a_new = lpu.calc_new_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values,
        c_1=c_1_n, c_2=c_2_n
    )
    
    calced_a_2db = lpu.calc_2d_box_sheath_expansion_param(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
        plot_ds.recession.values, plot_ds.theta_p_rads.values,
        c_1=c_1_b, c_2=c_2_b
    )

    ax.errorbar(dummy_theta, calced_a, label=r'old', fmt='-', color=cb_palette[5], linewidth=0.8, alpha=0.6)
    ax.errorbar(dummy_theta, calced_a_new, label=r'new', fmt='-', color=cb_palette[6], linewidth=0.8, alpha=0.6)
    ax.errorbar(dummy_theta, calced_a_2db, label=r"pete's", fmt='-', color=cb_palette[7], linewidth=0.8, alpha=0.6)
    ax.set_ylim(1e-3, 3.5e-1)
    ax.set_xlim(-0.5, 30.5)
#     ax.set_title(r'$c_1$ = {}, $c_2$ = {}'.format(c_1, c_2))

    ax.legend()

axes[1,1].set_ylim(1e-3, 5e-1)
# fig.suptitle(r'$c_1$ = {}, $c_2$ = {}'.format(c_1, c_2))
    
fig.tight_layout(rect=[0, 0, 1, 0.97])

In [92]:
fig, axes = plt.subplots(3, figsize=[6,8])

sheath_exp_funcs = [
    lpu.calc_sheath_expansion_coeff,
    lpu.calc_new_sheath_expansion_param,
    lpu.calc_2d_box_sheath_expansion_param,
]

probes_oi = [p for p in probe_desig.keys()]

for j, ax in enumerate(axes):
    for i, probe in enumerate(probes_oi):
        plot_ds = combined_ds.sel(**probe_desig[probe])

        ax.errorbar(plot_ds['theta'].values, plot_ds['ion_a'].values, yerr=plot_ds['ion_d_a'].values, marker='o', mfc='none', linestyle='none', 
                    label=f'{probe}', color=cb_palette[2*i], ecolor=cb_palette[2*i])
        
        if j == 0:
            calced_a = sheath_exp_funcs[j](
                T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
                c_1=0.9, c_2=0.9
            )
        else:
            calced_a = sheath_exp_funcs[j](
                T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
                plot_ds.recession.values, plot_ds.theta_p_rads.values, c_1=0.9, c_2=0.9,
            )

        ax.errorbar(dummy_theta, calced_a, label=r'calc', fmt='-', color=cb_palette[2*i], linewidth=0.8, alpha=0.6)
        # ax.errorbar(dummy_theta, flush_calced_a, label=r'calc', fmt='-', color='b', linewidth=0.8, alpha=0.6)
        ax.set_ylim(1e-3, 2.5e0)
        ax.set_xlim(-0.5, 30.5)

        ax.legend()
        
fig.tight_layout()

In [79]:
probe_inputs = []
for i in range(len(scans)):
    spl_path = spice_dir / all_run_dirs[scans[i]][0]
    probe_inputs.append(next(next(spl_path.glob('backup*')).glob('input.inp')))
    
# probe_inputs

fig, axes = plt.subplots(2, 2, figsize=[8,4], sharex=True, sharey=True)

for i, probe_inp in enumerate(probe_inputs):
    ax = axes[i%2][i//2]
    spu.plot_2d_sim_window(probe_inp, ax=ax, colour=cb_palette[i])
    ax.set_ylim(0,222)
    ax.set_xlim(0,555) #759)
    
fig.tight_layout()

{'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>], 'triangle': [<Section: triangle0>], 'circle': []}
{'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>], 'triangle': [<Section: triangle0>], 'circle': []}
{'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>], 'triangle': [], 'circle': []}
{'rectangle': [<Section: rectangle0>, <Section: rectangle1>, <Section: rectangle2>], 'triangle': [], 'circle': []}


---
## Normalised IV plots

In [28]:
def a_0(theta):
    return lpu.calc_sheath_expansion_coeff(
        T_e, 
        n_0, 
        L * lambda_D, 
        g * lambda_D, 
        np.radians(theta),
        c_1=0.5, 
        c_2=0.6
    )

In [37]:
fig, ax = plt.subplots(3, sharex=True, figsize=[6,10])

probes_oi = [p for p in probe_desig.keys() if 'recessed' not in p]

for i, probe in enumerate(probes_oi):
    dummy_voltage = np.linspace(-30, 0, 9950)
    piv_fitter = fts.PartialIVFitter()
    ax[i].plot(dummy_voltage, -piv_fitter.fit_function(dummy_voltage, *[1, a_0(90.0), 1.0]), 
               color='k', linestyle='--', label=f'Ideal')
    ax[i].plot(dummy_voltage, -piv_fitter.fit_function(dummy_voltage, *[1, a_0(10.0), 1.0]), 
               color='silver', linestyle='--', alpha=0.7, label=f'Ideal 10')
    
    for theta in combined_ds.theta.values:
        plot_ds = combined_ds.sel(**probe_desig[probe], theta=theta)

        V_f = fts.IVFitter.find_floating_pot(plot_ds['voltage'][:-20], plot_ds['current'][:-20])
        plot_ds = plot_ds.sel(voltage=slice(-9.95, V_f))

        voltage = (plot_ds['voltage'].values - V_f)[::2]
        current = plot_ds['current'].values[::2]
        d_current = plot_ds['d_current'].values[::2]
        
        I_0 = plot_ds.norm_iv_I_sat.values
        theta_label = plot_ds.theta_p.values + theta

        ax[i].errorbar(voltage, -current/I_0, yerr=d_current/I_0, fmt='.', mfc='none', markersize=5, 
                       label=f'{theta_label} - a={plot_ds.ion_a.values:.2g}')
        ax[i].legend()
        ax[i].set_title(probe) 
        
#         ax[i].set_ylim(-1.7,0.0)
        ax[i].set_xlim(-8.0,0.0)

fig.tight_layout()

---
### Decomposition of sheath expansion param plots
These can be categorised into 3 modes:
1. The original formula from Sam/Bergmann (no label)
2. My rederivation (new)
3. Pete's 2d box derivation (2d box)


In [109]:
decomp_modes = [
    None, 
    decompose_sheath_exp_param, 
    decompose_new_sheath_exp_param, 
    decompose_2d_box_sheath_exp_param,
    decompose_alt_new_sheath_exp_param
]
mode_labels = [
    None, 
    (r'$\cot{\theta}$', r'$a\sin^{1/2}{\theta}\cdot[L + g]$'),
    (r'$\tan{\theta} + 2\tan{\theta_p}$', r'$a\sin^{1/2}{\theta}[(L+g)\tan{\theta} + L\tan{\theta_p} - d_{\perp}]$'),
    (r'$\cot{\theta_{tot}}$', r'$a\sin^{1/2}{\theta_{tot}}[L_{eff} + \Delta_0 \cot{\theta_{tot}}]$'),
    (r'$\cot{\theta}$', r'$a\sin^{1/2}{\theta}[L + g + (L\tan{\theta_p} - d_{\perp}) * \cot{\theta}]$'),
]
mode_constants = [
    None,
    ('y_0', 'm'),
    ('m', 'y_0'),
    ('y_0', 'm'),
    ('y_0', 'm'),
]

def plot_decomp(ds, sheath_label='ion_a', mode=1, fit_fl=True, ax=None, kwargs_for_plot={}, 
                kwargs_for_fitplot={}, colour='r', plot_label=None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.figure
    
    if mode in [1, 2, 3, 4]:
        x, y = decomp_modes[mode](ds[sheath_label], ds['theta_rads'], L, g, 
                                  ds['recession'], ds['theta_p_rads'])
    else:
        raise ValueError('Mode must be 1, 2 or 3')
    
    x = x[~np.isnan(y)]
    y = y[~np.isnan(y)]
    
    if plot_label is None:
        plot_label = sheath_label
    
    ax.errorbar(x, y, yerr=ds[sheath_label]*y, color=colour, label=plot_label, **kwargs_for_plot)
    
    sl_fitter = fts.StraightLineFitter()
    fit_data = sl_fitter.fit(x, y)
    if fit_fl:    
        c1, c2 = mode_constants[mode]
        fit_label = r'$c_1$ = {:.2g}, $c_2$ = {:.2g}'.format(fit_data.get_param(c1), fit_data.get_param(c2))
        ax.plot(*fit_data.get_fit_plottables(), color=colour, label=fit_label, 
                **kwargs_for_fitplot)
        
    ax.set_xlabel(mode_labels[mode][0])
    ax.set_ylabel(mode_labels[mode][1])
    ax.legend()
    
    return ax, fit_data
        

In [30]:
combined_ds

<xarray.Dataset>
Dimensions:               (recession: 2, theta: 9, theta_p: 2, voltage: 401)
Coordinates:
  * recession             (recession) float64 0.0 0.001
    theta_p_desc          (theta_p) object 'flat' 'angled'
  * theta_p               (theta_p) float64 0.0 10.0
  * theta                 (theta) float64 3.0 4.0 5.0 6.0 ... 12.0 20.0 30.0
  * voltage               (voltage) float64 -9.95 -9.9 -9.85 ... 9.95 10.0 10.05
    recession_desc        (recession) object 'flush' 'recessed'
    theta_p_rads          (theta_p) float64 0.0 0.1745
    theta_rads            (theta) float64 0.05236 0.06981 ... 0.3491 0.5236
Data variables:
    str_iv_chi2           (recession, theta_p, theta) float64 6.743e+03 ... 159.0
    str_iv_reduced_chi2   (recession, theta_p, theta) float64 43.5 ... 1.026
    str_iv_I_sat          (recession, theta_p, theta) float64 -11.35 ... -71.91
    str_iv_d_I_sat        (recession, theta_p, theta) float64 3.13 ... 1.296
    str_iv_a              (recession, theta_p, theta) float64 1.868e-15 ... 0.04223
    str_iv_d_a            (recession, theta_p, theta) float64 0.12 ... 0.005092
    str_iv_T_e            (recession, theta_p, theta) float64 1.995 ... 0.9402
    str_iv_d_T_e          (recession, theta_p, theta) float64 0.3035 ... 0.02061
    str_iv_V_f            (recession, theta_p, theta) float64 -4.721 ... -2.776
    str_iv_d_V_f          (recession, theta_p, theta) float64 0.05557 ... 0.006002
    str_iv_run_dir        (recession, theta_p, theta) object 'marconi/spice2/new_sheath_exp/flat_flush/alpha_yz_-3.0' ... 'marconi/spice2/new_sheath_exp/angled_recessed/alpha_yz_-30.0'
    str_iv_voltage_min    (recession, theta_p, theta) float64 -9.95 ... -9.95
    str_iv_voltage_max    (recession, theta_p, theta) float64 -2.05 ... -2.05
    norm_iv_chi2          (recession, theta_p, theta) float64 138.3 ... 153.5
    norm_iv_reduced_chi2  (recession, theta_p, theta) float64 1.281 ... 1.073
    norm_iv_I_sat         (recession, theta_p, theta) float64 -6.476 ... -72.86
    norm_iv_d_I_sat       (recession, theta_p, theta) float64 0.1268 ... 0.7461
    norm_iv_a             (recession, theta_p, theta) float64 0.1512 ... 0.04027
    norm_iv_d_a           (recession, theta_p, theta) float64 0.01033 ... 0.003633
    norm_iv_voltage_min   (recession, theta_p, theta) float64 -5.454 ... -7.219
    norm_iv_voltage_max   (recession, theta_p, theta) float64 -0.004398 ... -0.01927
    ion_chi2              (recession, theta_p, theta) float64 127.9 ... 196.4
    ion_reduced_chi2      (recession, theta_p, theta) float64 1.184 ... 1.373
    ion_I_sat             (recession, theta_p, theta) float64 6.465 ... 79.16
    ion_d_I_sat           (recession, theta_p, theta) float64 0.04734 ... 0.4406
    ion_a                 (recession, theta_p, theta) float64 0.1515 ... 0.01446
    ion_d_a               (recession, theta_p, theta) float64 0.004415 ... 0.00207
    ion_voltage_min       (recession, theta_p, theta) float64 0.01708 ... 0.05173
    ion_voltage_max       (recession, theta_p, theta) float64 3.569 ... 4.404
    current               (recession, theta_p, theta, voltage) float64 -10.29 ... 915.7
    current_e             (recession, theta_p, theta, voltage) float64 -0.02646 ... -918.8
    current_i             (recession, theta_p, theta, voltage) float64 10.32 ... 3.105
    d_current             (recession, theta_p, theta, voltage) float64 0.2479 ... 6.614
    d_current_e           (recession, theta_p, theta, voltage) float64 0.01171 ... 6.695
    d_current_i           (recession, theta_p, theta, voltage) float64 0.2477 ... 0.495
    time                  (recession, theta_p, theta, voltage) float64 0.0 ... 2.581

In [88]:
probe = 'flat_flush'

plot_ds = combined_ds.sel(**probe_desig[probe]) #, theta=slice(2,10))
plot_f_ds = flush_ds.sel(theta=slice(3,30))

kwargs_for_plot={
    'ls':'none', 
    'mfc':'none', 
    'marker': '.',
}
kwargs_for_fitplot={
    'ls':'none', 
    'mfc':'none', 
}

fig, ax = plt.subplots(3, figsize=[6,10])
_, fit_data_1i = plot_decomp(plot_ds, sheath_label='ion_a', mode=1, ax=ax[0], colour=cb_palette[0], 
                             kwargs_for_plot=kwargs_for_plot)
_, fit_data_1a = plot_decomp(plot_f_ds, sheath_label='a', mode=1, ax=ax[0], colour=cb_palette[1],
                             kwargs_for_plot=kwargs_for_plot)

_, fit_data_2i = plot_decomp(plot_ds, sheath_label='ion_a', mode=2, ax=ax[1], colour=cb_palette[0], 
                             kwargs_for_plot=kwargs_for_plot)
_, fit_data_2a = plot_decomp(plot_f_ds, sheath_label='a', mode=2, ax=ax[1], colour=cb_palette[1],
                             kwargs_for_plot=kwargs_for_plot)

_, fit_data_4i = plot_decomp(plot_ds, sheath_label='ion_a', mode=4, ax=ax[2], colour=cb_palette[0], 
                             kwargs_for_plot=kwargs_for_plot)
_, fit_data_4a = plot_decomp(plot_f_ds, sheath_label='a', mode=4, ax=ax[2], colour=cb_palette[1],
                             kwargs_for_plot=kwargs_for_plot)

fig.tight_layout()


In [43]:
print('Old Param (ion fit)')
print(f'c1 = {fit_data_1i.get_param("y_0"):.3g} ± {fit_data_1i.get_param_err("y_0"):.3g}')
print(f'c2 = {fit_data_1i.get_param("m"):.3g} ± {fit_data_1i.get_param_err("m"):.3g}')

print('\nNew Param (ion fit)')
print(f'c1 = {fit_data_2i.get_param("m"):.3g} ± {fit_data_2i.get_param_err("m"):.3g}')
print(f'c2 = {fit_data_2i.get_param("y_0"):.3g} ± {fit_data_2i.get_param_err("y_0"):.3g}')

print('\n\nOld Param (old norm iv fit)')
print(f'c1 = {fit_data_1a.get_param("y_0"):.3g} ± {fit_data_1a.get_param_err("y_0"):.3g}')
print(f'c2 = {fit_data_1a.get_param("m"):.3g} ± {fit_data_1a.get_param_err("m"):.3g}')

print('\nNew Param (old norm iv fit)')
print(f'c1 = {fit_data_2a.get_param("m"):.3g} ± {fit_data_2a.get_param_err("m"):.3g}')
print(f'c2 = {fit_data_2a.get_param("y_0"):.3g} ± {fit_data_2a.get_param_err("y_0"):.3g}')


Old Param (ion fit)
c1 = 6.32 ± 1.07
c2 = 0.439 ± 0.107

New Param (ion fit)
c1 = 3.88 ± 0.525
c2 = 0.802 ± 0.134


Old Param (old norm iv fit)
c1 = 5.22 ± 0.515
c2 = 0.439 ± 0.0674

New Param (old norm iv fit)
c1 = 5.95 ± 0.419
c2 = 0.3 ± 0.118


In [111]:

probe_plot_dss = [combined_ds.sel(**probe_desig[list(probe_desig.keys())[i]]) for i in range(4)]


kwargs_for_plot={
    'ls':'none', 
    'mfc':'none', 
    'marker': '.',
}
kwargs_for_fitplot={
    'ls':'none', 
    'mfc':'none', 
}

fig, ax = plt.subplots(3, 2, figsize=[10,10])

for i in range(4):
    probe = list(probe_desig.keys())[i]
    print(probe)
    
    plot_decomp(probe_plot_dss[i], sheath_label='ion_a', mode=1, ax=ax[0,i//2], colour=cb_palette[i], 
                kwargs_for_plot=kwargs_for_plot, plot_label=probe)
    
    plot_decomp(probe_plot_dss[i], sheath_label='ion_a', mode=2, ax=ax[1,i//2], colour=cb_palette[i], 
                kwargs_for_plot=kwargs_for_plot, plot_label=probe)
    
    plot_decomp(probe_plot_dss[i], sheath_label='ion_a', mode=3, ax=ax[2,i//2], colour=cb_palette[i], 
                kwargs_for_plot=kwargs_for_plot, plot_label=probe)



fig.tight_layout()

angled_flush
angled_recessed
flat_flush
flat_recessed


In [59]:
# probe = 'angled'
# For all probes using the 

probes_oi = [p for p in probe_desig.keys()]

fig, ax = plt.subplots(2, 2, figsize=[10,10], sharey='row', sharex='row')


for i, probe in enumerate(probes_oi):
    plot_ds = combined_ds.sel(**probe_desig[probe], theta=slice(2,30))

#     x, y = decompose_2d_box_sheath_exp_param(plot_ds['ion_a'], plot_ds['theta_rads'], L, g, 
#                                              plot_ds['recession'], plot_ds['theta_p_rads'])
    x, y = decompose_sheath_exp_param(plot_ds['ion_a'], plot_ds['theta_rads'] + plot_ds['theta_p_rads'], L, g)                                     
    x = x[~np.isnan(y)]
    y = y[~np.isnan(y)]
    
#     calced_a = lpu.calc_2d_box_sheath_expansion_param(
#         T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), 
#         plot_ds.recession.values, plot_ds.theta_p_rads.values, c_1=0.5, c_2=0.6, delta_0=0.0
#     )
#     x_calc, y_calc = decompose_2d_box_sheath_exp_param(calced_a, np.radians(dummy_theta), L, g, 
#                                                        plot_ds.recession.values, plot_ds.theta_p_rads.values)
    calced_a = lpu.calc_sheath_expansion_coeff(
        T_e, n_0, L * lambda_D, g * lambda_D, np.radians(dummy_theta), c_1=0.5, c_2=0.6
    )
    x_calc, y_calc = decompose_sheath_exp_param(calced_a, np.radians(dummy_theta), L, g)

    sl_fitter = fts.StraightLineFitter()
    sl_fit_data = sl_fitter.fit(x, y)
    
    j = i // 2
    k = i % 2
    ax[j,k].plot(x, y, '.', color=probe_colour[probe])
    ax[j,k].plot(*sl_fit_data.get_fit_plottables(), label=probe, color=probe_colour[probe],
                 alpha=0.8)
    ax[j,k].plot(x_calc, y_calc, '--', color='k', linewidth=0.8, alpha=0.6, label='Theoretical')
    ax[j,k].legend()
    
#     ax[j,k].set_xlim(0.0, 1.0)
#     ax[j,k].set_ylim(0.0, 4.5)

    ax[0,k].set_xlim(1.0, 5.5)
    ax[0,k].set_ylim(1.0, 16.0)
    ax[1,k].set_xlim(1.0, 21.0)
    ax[1,k].set_ylim(1.0, 60.0)
#     ax[2,k].set_xlim(1.0, 60.0)
#     ax[2,k].set_ylim(1.0, 42.0)
    
    ax_theta = ax[j,k].twiny()
    x12 = ax[j,k].get_xlim()
    converted_theta = np.degrees(np.arctan(1/np.array(x12)) - plot_ds['theta_p_rads'].values)
    ax_theta.set_xlim(*converted_theta)
    
    
fig.tight_layout()